<a href="https://colab.research.google.com/github/HuyenTrang457/MCM/blob/main/pre_processing_for_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
file_path = "/content/Wimbledon_featured_matches1.csv"
data = pd.read_csv(file_path)
# bộ data đã lọc outlier từ pre_preocessing model 1

In [ ]:
# 2. Xử lý các giá trị chuỗi trong p1_score và p2_score
def convert_score_to_numeric(score):
    try:
        return float(score)
    except ValueError:
        return 50  # Chuyển giá trị "AD" hoặc bất kỳ chuỗi nào khác thành 50
data['p1_score'] = data['p1_score'].apply(convert_score_to_numeric)
data['p2_score'] = data['p2_score'].apply(convert_score_to_numeric)



In [ ]:
# Convert elapsed_time to seconds if column exists
def convert_elapsed_time_reset(df):
    df['elapsed_time_seconds'] = df['elapsed_time'].apply(lambda row: sum(int(x) * 60 ** i for i, x in enumerate(reversed(row.split(':')))))
    return df

data = data.groupby('match_id').apply(convert_elapsed_time_reset)
data = data.drop(columns=['elapsed_time'])# Feature engineering: differences
# Sets difference
data['sets_diff'] = data['p1_sets'] - data['p2_sets']

# Games difference
data['games_diff'] = data['p1_games'] - data['p2_games']

# Score difference
data['score_diff'] = data['p1_score'] - data['p2_score']

# Points won difference
data['points_won_diff'] = data['p1_points_won'] - data['p2_points_won']

# Ace difference
data['ace_diff'] = data['p1_ace'] - data['p2_ace']

# Double fault difference
data['double_fault_diff'] = data['p1_double_fault'] - data['p2_double_fault']
data['distance_diff'] = data['p1_distance_run'] - data['p2_distance_run']

# Binary encoding
# Server (is player1)
data['server_is_p1'] = np.where(data['server'] == 1, 1, 0)

# Serve number (first serve)
data['first_serve'] = np.where(data['serve_no'] == 1, 1, 0)

# Point victor (did player1 win the point?)
data['point_victor_p1'] = np.where(data['point_victor'] == 1, 1, 0)

# Ratio calculations
# Game win ratio
data['p1_game_win_ratio'] = data['p1_games'] / (data['p1_games'] + data['p2_games'] + 1e-8)

# Set win ratio
data['p1_set_win_ratio'] = data['p1_sets'] / (data['p1_sets'] + data['p2_sets'] + 1e-8)

# Points won ratio
data['p1_point_win_ratio'] = data['p1_points_won'] / (data['p1_points_won'] + data['p2_points_won'] + 1e-8)

# Ace ratio
data['p1_ace_ratio'] = data['p1_ace'] / (data['p1_ace'] + data['p2_ace'] + 1e-8)

# Double fault ratio
data['p1_double_fault_ratio'] = data['p1_double_fault'] / (data['p1_double_fault'] + data['p2_double_fault'] + 1e-8)
categorical_features = ['server', 'serve_width', 'serve_depth', 'return_depth']
columns_to_convert = ['serve_width', 'serve_depth', 'return_depth']
data[columns_to_convert] = data[columns_to_convert].applymap(lambda x: 1 if x == "TRUE" else 0)
# Normalize continuous features




In [ ]:
print(data.head(5))

                                  match_id         player1        player2  \
match_id                                                                    
2023-wimbledon-1301 0  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry   
                    1  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry   
                    2  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry   
                    3  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry   
                    4  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry   

                       set_no  game_no  point_no  p1_sets  p2_sets  p1_games  \
match_id                                                                       
2023-wimbledon-1301 0       1        1         1        0        0         0   
                    1       1        1         2        0        0         0   
                    2       1        1         3        0        0         0   
                    3       1        1         4        0   

In [ ]:
continuous_features = [
    'score_diff', 'sets_diff', 'games_diff', 'elapsed_time_seconds', 'p1_ace_ratio',
    'p1_double_fault_ratio', 'p1_game_win_ratio', 'points_won_diff'
]

# Handle missing values before scaling
imputer = SimpleImputer(strategy='mean')
data[continuous_features] = imputer.fit_transform(data[continuous_features])

# Standardize continuous features
scaler = StandardScaler()
data[continuous_features] = scaler.fit_transform(data[continuous_features])


# Save processed data
data.to_csv('processed_tennis_data_updated.csv', index=False)
print("Dữ liệu đã được xử lý và lưu thành công!")

Dữ liệu đã được xử lý và lưu thành công!
